In [3]:
import pymysql
import pandas as pd
import classified_names

In [4]:
#Dictionary for gender mapping
name_gender = classified_names.output
name_gender['Ararat'] = 'Male'

In [5]:
#Connecting to database
user = 'Copa'
password = 'COPA888'
host = 'localhost'
port = 3306
database = 'spyur_companies'
connection = pymysql.connect(user=user, password=password, host=host, port=port, database=database)

In [6]:
#Reading sql tables
company_t = "select * from company_table"
exec_t = 'select * from executive_table'
act_t = 'select * from activity_table'
prod_t = 'select * from product_table'
company_table = pd.read_sql(company_t, connection)
executive_table = pd.read_sql(exec_t, connection)
activity_table = pd.read_sql(act_t, connection)
product_table = pd.read_sql(prod_t, connection)

/tmp/ipykernel_267673/1341457086.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  company_table = pd.read_sql(company_t, connection)
/tmp/ipykernel_267673/1341457086.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  executive_table = pd.read_sql(exec_t, connection)
/tmp/ipykernel_267673/1341457086.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  activity_table = pd.read_sql(act_t, connection)
/tmp/ipykernel_267673/1341457086.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or databa

In [7]:
#removing empty executives, keeping only companies with one executive
executive_table = executive_table[executive_table['executive'] != 'N/A']
executive_counts = executive_table.groupby('url_id').size()
single_executive_companies = executive_counts[executive_counts == 1]
executive_table = executive_table[executive_table['url_id'].isin(single_executive_companies.index)]

In [8]:
#Splitting executive rows into full name and role columns
executive_table['executive'] = executive_table['executive'].astype('str')
executive_table['role'] = executive_table['executive'].str.split(',', n=1).str[1].str.strip()
executive_table['full_name'] = executive_table['executive'].str.split(',', n=1).str[0].str.strip()

In [9]:
#Function to extract first names for mapping
def extract_first_name(full_name, suffixes=["yan", "ian", "ts", "unc", "jan", "tz", "ians"]):
    if not full_name:
        return None
    
    name_parts = full_name.split()
    current_first_names = []

    for part in name_parts:
        if part.endswith('.'):
            continue  
        elif any(part.endswith(suffix) for suffix in suffixes):
            continue  
        else:
            current_first_names.append(part)

    return ' '.join(current_first_names) if current_first_names else None

executive_table['first_name'] = executive_table['full_name'].apply(extract_first_name)

In [10]:
# Removing rows where first_name is None or empty, mapping gender
executive_table = executive_table[executive_table['first_name'].notna() & (executive_table['first_name'] != '')]
executive_table['gender'] = executive_table['first_name'].map(name_gender)


In [11]:
executive_table

,url_id,executive_id,executive,url,role,full_name,first_name,gender
2,000c418c4d41709259a5d052890adf8d,388f0faf29a75c46749fdeb969f7a631,"Eduard K. Bezoyan, General Manager",https://www.spyur.am/en/companies/chanaparh-ro...,General Manager,Eduard K. Bezoyan,Eduard,Male
3,000d3c03f88ea054e58c29425b38e7bd,e72fc7c220c839ab242ad956a0ce2d42,"Armen Kostanyan, Director",https://www.spyur.am/en/companies/apex-medical...,Director,Armen Kostanyan,Armen,Male
5,000f2231367c8474d0c3f4f59cf74091,0739215b57c03e062976bbcc7253ba1c,"Haykush Barseghyan, Director",https://www.spyur.am/en/companies/vardenis-bas...,Director,Haykush Barseghyan,Haykush,Female
8,00206bf37e84a3dd752032dfdfd536e4,c117ba19c62d6104d16649c7e9f61916,"Varuzhan Vardanyan, Chairman",https://www.spyur.am/en/companies/panarmenian-...,Chairman,Varuzhan Vardanyan,Varuzhan,Male
9,0024bb29342551407e6e075391a3efc6,8b8252cc03073db35d21137a183f539e,"Anahit Basentsyan, Director",https://www.spyur.am/en/companies/yerevan-nurs...,Director,Anahit Basentsyan,Anahit,Female
...,...,...,...,...,...,...,...,...
22981,ffe8bd1cc946e3cbe9b56d3f2e26c015,5f8a6cb62eb3865e325f905a1363a1b9,"Lilit Yu. Zakaryan, Director",https://www.spyur.am/en/companies/arshaluis-gy...,Director,Lilit Yu. Zakaryan,Lilit,Female
22982,ffe8f6ad2bd0b40bb089fc04076d7b03,0095131c8a53cacfb2422808c95146d4,"Lilit K. Yesayan, Director",https://www.spyur.am/en/companies/de-fonseca-i...,Director,Lilit K. Yesayan,Lilit,Female
22983,ffea648ec8ade051c3276b49bddb6a21,95a60c82e0ea7811ae4bf9b5cc373bf1,"Georgy Manasyan, Director",https://www.spyur.am/en/companies/pinocchio-ki...,Director,Georgy Manasyan,Georgy,Male
22984,ffeab39fa4c38293865d28fa8bd57f4c,f18f0c412aee290ae2b050195bd9f3f9,"Anush Gijlaryan, Director",https://www.spyur.am/en/companies/maga-dance-s...,Director,Anush Gijlaryan,Anush,Female


In [12]:
executive_table['gender'].value_counts()

gender
Male       8619
Female     4357
Unknown     152
Name: count, dtype: int64

In [13]:
executive_table = executive_table[executive_table['gender']!= 'Unknown']

In [14]:
executive_table

,url_id,executive_id,executive,url,role,full_name,first_name,gender
2,000c418c4d41709259a5d052890adf8d,388f0faf29a75c46749fdeb969f7a631,"Eduard K. Bezoyan, General Manager",https://www.spyur.am/en/companies/chanaparh-ro...,General Manager,Eduard K. Bezoyan,Eduard,Male
3,000d3c03f88ea054e58c29425b38e7bd,e72fc7c220c839ab242ad956a0ce2d42,"Armen Kostanyan, Director",https://www.spyur.am/en/companies/apex-medical...,Director,Armen Kostanyan,Armen,Male
5,000f2231367c8474d0c3f4f59cf74091,0739215b57c03e062976bbcc7253ba1c,"Haykush Barseghyan, Director",https://www.spyur.am/en/companies/vardenis-bas...,Director,Haykush Barseghyan,Haykush,Female
8,00206bf37e84a3dd752032dfdfd536e4,c117ba19c62d6104d16649c7e9f61916,"Varuzhan Vardanyan, Chairman",https://www.spyur.am/en/companies/panarmenian-...,Chairman,Varuzhan Vardanyan,Varuzhan,Male
9,0024bb29342551407e6e075391a3efc6,8b8252cc03073db35d21137a183f539e,"Anahit Basentsyan, Director",https://www.spyur.am/en/companies/yerevan-nurs...,Director,Anahit Basentsyan,Anahit,Female
...,...,...,...,...,...,...,...,...
22981,ffe8bd1cc946e3cbe9b56d3f2e26c015,5f8a6cb62eb3865e325f905a1363a1b9,"Lilit Yu. Zakaryan, Director",https://www.spyur.am/en/companies/arshaluis-gy...,Director,Lilit Yu. Zakaryan,Lilit,Female
22982,ffe8f6ad2bd0b40bb089fc04076d7b03,0095131c8a53cacfb2422808c95146d4,"Lilit K. Yesayan, Director",https://www.spyur.am/en/companies/de-fonseca-i...,Director,Lilit K. Yesayan,Lilit,Female
22983,ffea648ec8ade051c3276b49bddb6a21,95a60c82e0ea7811ae4bf9b5cc373bf1,"Georgy Manasyan, Director",https://www.spyur.am/en/companies/pinocchio-ki...,Director,Georgy Manasyan,Georgy,Male
22984,ffeab39fa4c38293865d28fa8bd57f4c,f18f0c412aee290ae2b050195bd9f3f9,"Anush Gijlaryan, Director",https://www.spyur.am/en/companies/maga-dance-s...,Director,Anush Gijlaryan,Anush,Female


In [16]:
#Saving table as pkl file
executive_table.to_pickle('executive_table.pkl')
company_table.to_pickle('company_table.pkl')
activity_table.to_pickle('activity_table.pkl')
product_table.to_pickle('product_table.pkl')